In [8]:
from bs4 import BeautifulSoup
import datetime
import pickle
import requests
import random

In [9]:
def getNewsForWeek(date):
    thedate = date
    print("important:" + thedate.strftime('%Y-%m-%d'))
    file = open('data/news/' + thedate.strftime('%Y-%m-%d') + '.csv', 'w')
    file.write("Ticker,Headlines")
    file.write('\n')

    print("FILE:" + 'data/news/' + thedate.strftime('%Y-%m-%d') + '.csv', 'w')
    for i in range(len(stocks)):
        allnews = ""
        for j in range(14):
            thedate = date
            thedate += datetime.timedelta(days=j)
            #print('Getting news for ' + thedate.strftime('%Y-%m-%d'))
            query = 'http://www.reuters.com/finance/stocks/company-news/' + stocks[i] + '.O' + '?date=' + format(thedate.month, '02d') + format(thedate.day, '02d') + str(thedate.year)
            #print(query)
            #print('Getting news for ' + stocks[i])

            response = requests.get(query)
            soup = BeautifulSoup(response.text, "html.parser")
            divs = soup.findAll('div', {'class': 'feature'})
            divs2 = soup.findAll('div', {'class': 'topStory'})
            #print('Found ' + str(len(divs) + len(divs2)) + ' articles.')


            if(len(divs) == 0 and len(divs2) == 0):
                continue
            #data = u'' 
            data = []


            for div in divs:
                test = div.findAll(text=True)[0]
                data = data + [test]
                #data = data.join(div.findAll(text=True))
            #news = data.split('\n')[0]
            #news = news.strip().replace(',','')
            news = data

            data2 = u''

            for div in divs2:
                data2 = data2.join(div.findAll(text=True))

            news2 = data2
            news2 = news2.replace('\n', "--")
            test = news2.split("--")
            for j in range(len(test)):
                if test[j] != "" and test[j] != "Continue Reading" :
                    news2 = test[j]
                    break

            news = news + [news2]
            for k in range(len(news)):
                news[k] = news[k].replace(",", "")
            news = "|".join(news) 
            #print(news)
            #news = data.strip().split('\n')
            #news = data.strip().replace('\n',' ').split(' ')
            #news = " ".join(news)
            allnews = allnews + "|" + news
    

        #print(allnews)
        file.write(stocks[i] +   ',' + allnews[1:])
        file.write('\n')
        thedate += datetime.timedelta(days=-14)
    file.close()


In [28]:
def getNews():
    endDate = datetime.date.today()
    date = endDate
    date += datetime.timedelta(days=-365 * 7 + -337)
    print(date)
    while(date <= endDate):
        getNewsForWeek(date)
        date += datetime.timedelta(days=14)



In [30]:
FNAME = "snp500_formatted.txt"
stocks = []
with open(FNAME) as f:
    stocks = f.readlines()
for i in range(len(stocks)):
    stocks[i] = stocks[i].rstrip('\n')
#stocks = ["MSFT", "GOOGL", "AAPL", "TSLA", "WDC", "RAD", "BAC", "F", "GE", "T", "M", "WFC", "JCP", "KR", "GM", "KO", "DIS", "SWN", "PFE", "WMT" ]
stocks = ['AAPL', 'FB', 'GOOGL', 'TSLA', 'AMZN', 'TSLA', 'BKRB', 'JNJ']
getNews()

2010-01-01
important:2010-01-01
FILE:data/news/2010-01-01.csv w
important:2010-01-15
FILE:data/news/2010-01-15.csv w
important:2010-01-29
FILE:data/news/2010-01-29.csv w
important:2010-02-12
FILE:data/news/2010-02-12.csv w
important:2010-02-26
FILE:data/news/2010-02-26.csv w
important:2010-03-12
FILE:data/news/2010-03-12.csv w
important:2010-03-26
FILE:data/news/2010-03-26.csv w
important:2010-04-09
FILE:data/news/2010-04-09.csv w
important:2010-04-23
FILE:data/news/2010-04-23.csv w
important:2010-05-07
FILE:data/news/2010-05-07.csv w
important:2010-05-21
FILE:data/news/2010-05-21.csv w
important:2010-06-04
FILE:data/news/2010-06-04.csv w
important:2010-06-18
FILE:data/news/2010-06-18.csv w
important:2010-07-02
FILE:data/news/2010-07-02.csv w
important:2010-07-16
FILE:data/news/2010-07-16.csv w
important:2010-07-30
FILE:data/news/2010-07-30.csv w
important:2010-08-13
FILE:data/news/2010-08-13.csv w
important:2010-08-27
FILE:data/news/2010-08-27.csv w
important:2010-09-10
FILE:data/news

important:2015-12-11
FILE:data/news/2015-12-11.csv w
important:2015-12-25
FILE:data/news/2015-12-25.csv w
important:2016-01-08
FILE:data/news/2016-01-08.csv w
important:2016-01-22
FILE:data/news/2016-01-22.csv w
important:2016-02-05
FILE:data/news/2016-02-05.csv w
important:2016-02-19
FILE:data/news/2016-02-19.csv w
important:2016-03-04
FILE:data/news/2016-03-04.csv w
important:2016-03-18
FILE:data/news/2016-03-18.csv w
important:2016-04-01
FILE:data/news/2016-04-01.csv w
important:2016-04-15
FILE:data/news/2016-04-15.csv w
important:2016-04-29
FILE:data/news/2016-04-29.csv w
important:2016-05-13
FILE:data/news/2016-05-13.csv w
important:2016-05-27
FILE:data/news/2016-05-27.csv w
important:2016-06-10
FILE:data/news/2016-06-10.csv w
important:2016-06-24
FILE:data/news/2016-06-24.csv w
important:2016-07-08
FILE:data/news/2016-07-08.csv w
important:2016-07-22
FILE:data/news/2016-07-22.csv w
important:2016-08-05
FILE:data/news/2016-08-05.csv w
important:2016-08-19
FILE:data/news/2016-08-19